In [1]:
import numpy as np

In [2]:
t = [20, 21, 22, 23, 24, 25]
d = [0.99823, 0.998019, 0.997797, 0.997565, 0.997323, 0.997071]
n = len(t)

In [3]:
"""
Интерполяция методом Лагранжа
"""
def lagrange(n, t, d, *x):
    for k in range(len(x)):
        result = 0
        for i in range(n - 1):
            tmp = 1
            for j in range(n - 1):
                if i != j:
                    tmp *= (x[k] - t[j]) / (t[i] - t[j])
            tmp *= d[i]
            result += tmp
        print("t =", x[k], " --->  D =" ,result)

lagrange(n,t,d, 21.5, 22.7, 24.3)

t = 21.5  --->  D = 0.9979092890625
t = 22.7  --->  D = 0.9976356306625
t = 24.3  --->  D = 0.9972483266625001


In [4]:
"""
Интерполяция методом Ньютона
"""
def dd(x1, x2):
    return (d[x2] - d[x1]) / (x2 - x1)

f_x0x1x2 = (dd(1,2) - dd(0,1)) / (t[2] - t[0])
f_x1x2x3 = (dd(2,3) - dd(1,2)) / (t[3] - t[1])
f_x2x3x4 = (dd(3,4) - dd(2,3)) / (t[4] - t[2])
f_x3x4x5 = (dd(4,5) - dd(3,4)) / (t[5] - t[3])

f_x0x1x2x3 = (f_x1x2x3 - f_x0x1x2) / (t[3] - t[0])
f_x1x2x3x4 = (f_x2x3x4 - f_x1x2x3) / (t[4] - t[1])
f_x2x3x4x5 = (f_x3x4x5 - f_x2x3x4) / (t[5] - t[2])

f_x0x1x2x3x4 = (f_x1x2x3x4 - f_x0x1x2x3) / (t[4] - t[0])
f_x1x2x3x4x5 = (f_x2x3x4x5 - f_x1x2x3x4) / (t[5] - t[1])

def newton(x):
    newton_x = d[0] + dd(0,1) * (x - t[0]) + \
               f_x0x1x2 * (x - t[0]) * (x - t[1]) + \
               f_x0x1x2x3 * (x - t[0]) * (x - t[1]) * (x - t[2]) + \
               f_x0x1x2x3x4 * (x - t[0]) * (x - t[1]) * (x - t[2]) * (x - t[3]) + \
               ((f_x1x2x3x4x5 - f_x0x1x2x3x4) / (t[5] - t[0])) * (x - t[0]) * \
                (x - t[1]) * (x - t[2]) * (x - t[3]) * (x - t[4])
    return newton_x

x = [21.5, 22.7, 24.3]

for x0 in x:
    print('t =', x0, '---> D = ', newton(x0))

t = 21.5 ---> D =  0.99790927734375
t = 22.7 ---> D =  0.99763564110475
t = 24.3 ---> D =  0.99724843273275


In [5]:
"""
Интерполяция кубическим сплайном
"""
def decLU3(a, b, c):
    n = len(a)
    d = np.copy(a)
    u = np.copy(b)
    l = np.copy(c)
    for i in range(1, n):
        al = l[i] / d[i - 1]
        d[i] = d[i] - al*u[i - 1]
        l[i] = al
    return d, u, l

def solveLU3(a, b, c, f):
    n = len(a)
    d, u, l = decLU3(a, b, c)
    x = np.copy(f)
    for i in range(1, n):
        x[i] = x[i] - l[i] * x[i-1]
    x[n-1] = x[n-1] / d[n-1]
    for i in range(n-2, -1, -1):
        x[i] = (x[i] - u[i]*x[i+1]) / d[i]
    return x

def coefspline(x, y):
    n = len(x) - 1
    a = np.ones((n+1), 'float')
    b = np.zeros((n+1), 'float')
    c = np.zeros((n+1), 'float')
    f = np.zeros((n+1), 'float')
    for i in range(1, n):
        a[i] = 2.*(x[i+1] - x[i-1])
        b[i] = x[i+1] - x[i]
        c[i] = x[i] - x[i - 1]
        f[i] = 6.*(y[i+1] - y[i]) / (x[i+1] - x[i]) \
              - 6. * (y[i] - y[i-1]) / (x[i] - x[i - 1])
    return solveLU3(a, b, c, f)

def spline(x, y, c, x0):
    def find(x, x0):
        iL = 0
        iR = len(x) - 1
        while iR - iL > 1:
            i = int((iL + iR) / 2)
            if x0 < x[i]:
                iR = i
            else:
                iL = i
        return iL
    i = find(x, x0)
    h = x[i+1] - x[i]
    y0 = ((x[i+1] - x0)**3/h - (x[i+1] - x[0])*h)*c[i]/6. \
       + ((x[0] - x[i])**3 / h - (x0 - x[i])*h) * c[i+1] /6. \
       + (y[i]*(x[i+1] - x0) + y[i+1]*(x0 - x[i]))
    return y0

x = np.array(t)
y = np.array(d)
x1 = np.array([21.5, 22.7, 24.3])
c = coefspline(x, y)
for i in x1:
    print("t =", i, " --->  D =" ,spline(x, y, c, i))

t = 21.5  --->  D = 0.9979147105263159
t = 22.7  --->  D = 0.9976529841866032
t = 24.3  --->  D = 0.9972571819282294
